In [2]:
from datetime import datetime, timedelta
import requests
from bs4 import BeautifulSoup
import csv
import xml.etree.ElementTree as ET
import time
import random 

import pandas as pd

from requests.auth import HTTPBasicAuth

In [3]:
epo_countries = [
    "AL",  # Albania
    "AT",  # Austria
    "BE",  # Belgium
    "BG",  # Bulgaria
    "CH",  # Switzerland
    "CY",  # Cyprus
    "CZ",  # Czech Republic
    "DE",  # Germany
    "DK",  # Denmark
    "EE",  # Estonia
    "ES",  # Spain
    "FI",  # Finland
    "FR",  # France
    "GB",  # United Kingdom
    "GR",  # Greece
    "HR",  # Croatia
    "HU",  # Hungary
    "IE",  # Ireland
    "IS",  # Iceland
    "IT",  # Italy
    "LI",  # Liechtenstein
    "LT",  # Lithuania
    "LU",  # Luxembourg
    "LV",  # Latvia
    "MC",  # Monaco
    "MK",  # North Macedonia
    "MT",  # Malta
    "NL",  # Netherlands
    "NO",  # Norway
    "PL",  # Poland
    "PT",  # Portugal
    "RO",  # Romania
    "RS",  # Serbia
    "SE",  # Sweden
    "SI",  # Slovenia
    "SK",  # Slovakia
    "SM",  # San Marino
    "TR",  # Türkiye
    "US",  # United States
    "CN",  # China
    "JP",  # Japan
    "KR",  # South Korea
    "IN",  # India
    "RU",  # Russia
    "AU",  # Australia
    "BR",  # Brazil
    "CA"   # Canada
]


In [4]:
consumer_key = 'Lq3V1hAxII2Fv8YSdcrhfA538HzhsfzhelYqSUgzGU0A8AEU'
consumer_secret_key = 'jHJjRSzVQBjCizDYmFg4cqWza2R8TOCcLaecRbXrJQcseLu8Lr0AqYXL1ldkAx8A'

# EPO OAuth2 token URL
token_url = "https://ops.epo.org/3.2/auth/accesstoken"

def get_token():
    # Request access token
    response = requests.post(
        token_url,
        auth=HTTPBasicAuth(consumer_key, consumer_secret_key),
        data={"grant_type": "client_credentials"},
    )
    
    # Extract token
    if response.status_code == 200:
        token = response.json()["access_token"]
        return token
    else:
        print("Failed to retrieve access token.")
        return None

In [5]:
#patent search without countries for faster collection (used up until 1866)
def patent_search(date_range, token, start, end):

    search_url = "https://ops.epo.org/rest-services/published-data/search"

    # Headers with authentication
    headers = {
        "Authorization": f"Bearer {token}",
        "Accept": "application/xml",
        "User-Agent": "ProjectAcademicBot/1.0 (for educational purposes only, contact: collen.ellis@sorbonne-universite.fr)",
        "Connection": "keep-alive"
    }
    
    params = {
    "q": f'pd={date_range}',
    "range": f"{start}-{end}",
    }
    
    response = requests.get(search_url, headers=headers, params=params)
    
    if response.status_code == 200:
        
        patents_list = []
        
        # Namespace dictionary (important for parsing)
        ns = {"epo": "http://www.epo.org/exchange"}
        
        xml_data = response.text
        root = ET.fromstring(xml_data)
        
        for patent in root.findall(".//epo:document-id", namespaces=ns):
            country = patent.find("epo:country", namespaces=ns)
            doc_number = patent.find("epo:doc-number", namespaces=ns)
        
            if country is not None and doc_number is not None:
                patents_list.append({
                "country": country.text,
                "doc_number": doc_number.text
            })
        
        return patents_list
    else:
        print(f"Error: {response.status_code}, {response.text}")
        return []

In [6]:
#patent search with countries (used after 1866 as more than 24k patents per year)
'''def patent_search(date_range, token, start, end, country):

    search_url = "https://ops.epo.org/rest-services/published-data/search"

    # Headers with authentication
    headers = {
        "Authorization": f"Bearer {token}",
        "Accept": "application/xml",
        "User-Agent": "ProjectAcademicBot/1.0 (for educational purposes only, contact: collen.ellis@sorbonne-universite.fr)",
        "Connection": "keep-alive"
    }
    
    params = {
    "q": f'pd={date_range} and pn={country}',
    "range": f"{start}-{end}",
    }
    
    response = requests.get(search_url, headers=headers, params=params)
    
    if response.status_code == 200:
        
        patents_list = []
        
        # Namespace dictionary (important for parsing)
        ns = {"epo": "http://www.epo.org/exchange"}
        
        xml_data = response.text
        root = ET.fromstring(xml_data)
        
        for patent in root.findall(".//epo:document-id", namespaces=ns):
            country = patent.find("epo:country", namespaces=ns)
            doc_number = patent.find("epo:doc-number", namespaces=ns)
        
            if country is not None and doc_number is not None:
                patents_list.append({
                "country": country.text,
                "doc_number": doc_number.text
            })
        
        return patents_list
    else:
        print(f"Error: {response.status_code}, {response.text}")
        return []'''

'def patent_search(date_range, token, start, end, country):\n\n    search_url = "https://ops.epo.org/rest-services/published-data/search"\n\n    # Headers with authentication\n    headers = {\n        "Authorization": f"Bearer {token}",\n        "Accept": "application/xml",\n        "User-Agent": "ProjectAcademicBot/1.0 (for educational purposes only, contact: collen.ellis@sorbonne-universite.fr)",\n        "Connection": "keep-alive"\n    }\n    \n    params = {\n    "q": f\'pd={date_range} and pn={country}\',\n    "range": f"{start}-{end}",\n    }\n    \n    response = requests.get(search_url, headers=headers, params=params)\n    \n    if response.status_code == 200:\n        \n        patents_list = []\n        \n        # Namespace dictionary (important for parsing)\n        ns = {"epo": "http://www.epo.org/exchange"}\n        \n        xml_data = response.text\n        root = ET.fromstring(xml_data)\n        \n        for patent in root.findall(".//epo:document-id", namespaces=ns):

In [ ]:
#code to scrape quarterly to allow quicker collection of years with lesser patents
quartiles = {
    "Q1": ("0101", "0331"),
    "Q2": ("0401", "0630"),
    "Q3": ("0701", "0930"),
    "Q4": ("1001", "1231"),
}

all_patents = []
range_size = 100
max_total = 2000 #EPO limit per request


for year in range(1861, 1862):
    token = get_token()
    year_results = []
    
    for quarter, (start_suffix, end_suffix) in quartiles.items():
        start = 1
        end = range_size
        date_range = f"{year}{start_suffix}-{year}{end_suffix}"
        
        while start <= max_total:
            #time.sleep(.5)
            #time.sleep(random.uniform(0,0.5))
            time.sleep(random.uniform(2, 4))
            query_results = patent_search(date_range, token, start, end)
            #print(query_results)
            if query_results is None:
                print("Error is none")
                break
            if len(query_results) < 100:
                year_results.extend(query_results)
                break
            
            year_results.extend(query_results)
            start += range_size
            end += range_size
    print(year, "has", len(year_results))        
    all_patents.append(year_results)

                
                
        


In [ ]:
months = {
    "01": ("0101", "0131"),
    "02": ("0201", "0228"),  # Not accounting for leap years
    "03": ("0301", "0331"),
    "04": ("0401", "0430"),
    "05": ("0501", "0531"),
    "06": ("0601", "0630"),
    "07": ("0701", "0731"),
    "08": ("0801", "0831"),
    "09": ("0901", "0930"),
    "10": ("1001", "1031"),
    "11": ("1101", "1130"),
    "12": ("1201", "1231"),
}

all_patents = []
range_size = 100
max_total = 2000  # EPO limit per request

for year in range(1867, 1900):
    token = get_token()
    year_results = []

    for month, (start_suffix, end_suffix) in months.items():
        start = 1
        end = range_size
        date_range = f"{year}{start_suffix}-{year}{end_suffix}"

        while start <= max_total:
            time.sleep(random.uniform(2, 4))
            query_results = patent_search(date_range, token, start, end)

            if query_results is None:
                print("Error is none")
                break
            if len(query_results) < 100:
                year_results.extend(query_results)
                break

            year_results.extend(query_results)
            start += range_size
            end += range_size

    print(year, "has", len(year_results))
    all_patents.append(year_results)


In [7]:
def generate_weekly_ranges(year):
    start_date = datetime(year, 1, 1)
    end_date = datetime(year, 12, 31)
    weekly_ranges = []

    current_start = start_date
    while current_start <= end_date:
        current_end = current_start + timedelta(days=6)
        if current_end > end_date:
            current_end = end_date

        start_str = current_start.strftime("%Y%m%d")
        end_str = current_end.strftime("%Y%m%d")
        weekly_ranges.append((start_str, end_str))

        current_start = current_end + timedelta(days=1)

    return weekly_ranges

In [46]:
all_patents = []
range_size = 100
max_total = 2000  # EPO limit per request

for year in range(1922, 1924):
    year_results = []

    weekly_ranges = generate_weekly_ranges(year)

    for start_date, end_date in weekly_ranges:
        token = get_token()
        start = 1
        end = range_size
        date_range = f"{start_date}-{end_date}"

        while start <= max_total:
            time.sleep(random.uniform(3,5))
            query_results = patent_search(date_range, token, start, end)

            if query_results is None:
                print("Error is none")
                break
            if len(query_results) < 100:
                year_results.extend(query_results)
                break

            year_results.extend(query_results)
            start += range_size
            end += range_size

    print(f"{year} has {len(year_results)} patents at {datetime.now().strftime('%H:%M')}")
    all_patents.append(year_results)

1922 has 103601 patents at 12:16
1923 has 103631 patents at 13:30


In [ ]:
def generate_3day_ranges(year):
    start_date = datetime(year, 1, 1)
    end_date = datetime(year, 12, 31)
    day_ranges = []

    current_start = start_date
    while current_start <= end_date:
        current_end = current_start + timedelta(days=2)  # 3-day block (start date + 2 days)
        if current_end > end_date:
            current_end = end_date

        start_str = current_start.strftime("%Y%m%d")
        end_str = current_end.strftime("%Y%m%d")
        day_ranges.append((start_str, end_str))

        current_start = current_end + timedelta(days=1)  # Move to the next block

    return day_ranges

In [20]:
all_patents = []
range_size = 100
max_total = 2000  # EPO limit per request

for year in range(1935, 1936):
    year_results = []

    # Generate 3-day ranges
    day_ranges = generate_3day_ranges(year)

    for start_date, end_date in day_ranges:
        token = get_token()
        start = 1
        end = range_size
        date_range = f"{start_date}-{end_date}"

        while start <= max_total:
            time.sleep(random.uniform(4, 5))
            query_results = patent_search(date_range, token, start, end)

            if query_results is None:
                print("Error is none")
                break
            if len(query_results) < 100:
                year_results.extend(query_results)
                break

            year_results.extend(query_results)
            start += range_size
            end += range_size

    print(f"{year} has {len(year_results)} patents at {datetime.now().strftime('%H:%M')}")
    all_patents.append(year_results)

KeyboardInterrupt: 

In [18]:
#start at 11:34
csv_filename = "patents_foster_1935-1935.csv"

with open(csv_filename, mode="w", newline="", encoding="utf-8") as csvfile:
    writer = csv.writer(csvfile)
    
    # Write header
    writer.writerow(["year", "country", "doc_number"])
    
    # Flatten the data while writing
    for i in range (len(all_patents)):
        year = 1935 + i
        patent_list = all_patents[i]

        for patent in patent_list:
            writer.writerow([year, patent["country"], patent["doc_number"]])

print(f"CSV file '{csv_filename}' has been created successfully.")

CSV file 'patents_foster_1935-1935.csv' has been created successfully.


In [44]:
import pandas as pd

# Load the CSV
df = pd.read_csv("patents_foster_1920-1921.csv")

# Increment the year column by 1
df["year"] = df["year"] + 4

# Save it back to the same file (or choose a different name if you want to keep the original)
df.to_csv("patents_foster_1920-1921.csv", index=False)

print("Years successfully shifted by +1.")


Years successfully shifted by +1.
